# Data Vizualiation and Interactive Data Displays


In this lab, we will explore some advance graphing and data display options, largely using `ggplot` and related packages. There is so much here; we could easily spend an entire semester learning this material. 

Please install the required packages (if not already installed) and load the packages. We will be examining the racial income gap that still exists in the United States after centures of both *de jure* and *de facto* racial segregation and discrimination.

In [ ]:
#install.packages('tidyverse')
#install.packages('tidycensus')
#install.packages('ggdensity')
#install.packages('ggdist')
#install.packages('scales')
#install.packages('ggThemes') # has some good color themes!
#install.packages('scico')    # has some good color themes!
#install.packages('ggprism')  # has some good color themes!
#install.packages('paletteer')

In [ ]:
library(tidyverse)
library(tidycensus)
library(ggdensity)
library(ggdist)
library(scales)
library(paletteer)
library(plotly)

## `tidycensus`

`tidycensus` is, for my money, one of the top R packages. It provides integration with the U.S. Census Bureau's API, allowing you to download data directly by your geography of interest into R. To do this, you need to know the Census product you want to use, the time, the level (state, county, zip code, etc), and the variables. 

In this example, we will use data from the American Community Survey, the 5-year estimates for 2023, which aggregate ACS data from 2019-2023. ACS data has many advantages over the decennial census data, offering additional questions about income and education that are not asked on the decennial census. The downside, of course, is that the ACS is a survey, and we cannot get reliable estimates of very small units (like census tracts or block groups) using these data. 

For this activity, let's download data on race, education, and income by micro/metropolitan statistical areas ("cbsa"). To use `tidycensus`, you first need to know the name of the census variable. I recommend using the `load_variables()` function to get a table of available variables, with labels, from the dataset. Then, you would need to carefully review the table to find the variables you want to download. Note that often census data comes as counts when we would typically want percentages or proportions. So be sure to collect both the count of the variable of interest (numerator) and the variable that captures the relevant population (denominator).

I *strongly* recommend reviewing `tidycensus`' [excellent website](https://walker-data.com/tidycensus/index.html) to better understand the core functions.

In [ ]:
# see what data are available and identify names
vars_2023 <- load_variables(year = 2023, dataset = "acs5")

# since there are many variables we want to download, store 
#   as a list here and then call up later.
varlist <- c(total.pop = "B03002_001" ,             # here I give new variable names.
             white.count = "B03002_003" ,           # the actual Census column name is in quotes 
             black.count = "B03002_004" ,
             latino.count = "B03002_012" ,
             medfaminc.black = "B19113B_001" ,
             medfaminc.white = "B19113H_001",
             edpop.male.black = "C15002B_002",      # note that the ed data is split by sex
             edpop.female.black = "C15002B_007",
             edpop.male.white = "C15002H_002",
             edpop.female.white = "C15002H_007",
             ba.male.black = "C15002B_006",
             ba.female.black = "C15002B_011",
             ba.male.white = "C15002H_006",
             ba.female.white = "C15002H_011")


df <- get_acs(geography = "cbsa",           # tells the geo level
              variables = varlist,          # plugs in our stored list
              year = 2023,                  # sets the year of the data
              sumfile = "acs5")             # data source

If you use `tidycensus` a lot, you may want to register with the Census Bureau to receive an api key, which will allow you to download more data in a day. 

Take a look your data:

In [ ]:
head(df, n = 20)

Variables are stored in rows, not columns. `GEOID` and `NAME` record the metropolitan area. `estimate` contains the data. `moe` is the margin of error. We need to convert these additional rows into columns. We can do that with `pivot_wider()` from the `tidyverse`'s `dplyr` package.

In [ ]:
df <- df |> select(-moe) |>                     # remove moe variable
            pivot_wider(names_from = variable,  # new variable names come from `variable`
                        values_from = estimate) # new data comes from `estimate`

head(df)

Much better! Now we have one observation per metro area, and each variable exists in its own column in our data frame. Let's continue processing our data by creating percentages and aggregating counts for men and women by race. Let's also create an ordinal measure of metro size that we will use later for graphing.

In [ ]:
df <- df |> mutate(ba.black = ((ba.male.black + ba.female.black)/
                               (edpop.male.black + edpop.female.black)),
                    ba.white = ((ba.male.white + ba.female.white)/
                                (edpop.male.white + edpop.female.white)),
                    white.pct = 100*white.count / total.pop,
                    black.pct = 100*black.count / total.pop,
                    pop3 = case_when(
                            total.pop < 100000 ~ 0,
                            total.pop >=100000 & total.pop < 1000000 ~ 1,
                            total.pop >= 1000000 ~ 2),
                            pop3 = ordered(pop3, labels = c("Small", "Medium", "Large"))
                    )

Let's start our work examining a simple scatterplot between graduation rates for Black Americans and median family income for Black Americans, by micropolitan/metropolitan statistical areas:

In [ ]:
ggplot(df, aes(x = ba.black, y = medfaminc.black, )) +
        geom_point()      

Ok. We can do better! Let's cover two quick changes. First, this graphic currently displays two variables: BA rate and median income, on the x and y axes, respectively. Spatial position is being used here to convey the relationship between the two variables. Let's now use point size to display a third variable. This is called a *bubble plot*. 

In [ ]:
p1 <- ggplot(df, aes(   x = ba.black, 
                        y = medfaminc.black, 
                        size = total.pop )) +  # changes size of dot according to size of total.pop
        geom_point(alpha = .2)                 # makes point 20% opaque

p1        

This is ok, but the points are all fairly similar, even though there is a wide variation in metro size. We can increase the variance in point size by using the `scale_size_area` argument.

In [ ]:
p2 <- p1 + scale_size_area(max_size = 20) # play around with different max sizes. 

In a Markdown block, evaluate this graph. What are the pros and cons of presenting the relationship between these two variables using this bubble plot with potential large bubbles compared with the previous two plots?

Let's continue refining this plot. We can remove the legend and add descriptive titles. 

In [ ]:
p2 + theme_minimal() +
     theme(legend.position = "none")  + # removes the legend
        labs(title = "In metro areas with a higher proportion of Black Americans \nholding college degrees, Black Americans have higher median \nincomes - especially in larger metro areas", # the \n inserts line breaks in the title
        x = "Proportion of Black Americans in Metro with College Degree",
        y = "Median Family Income of Black Americans")

This is a good step. How about we center the title and make the font bigger? 

In [ ]:
p2 + theme_minimal() +
     theme(legend.position = "none", 
            plot.title = element_text(size = 14, face = "bold", hjust = 0.5)) + # size arg sets font size, hjust is for horizontal adjustment
        labs(title = "In metro areas with a higher proportion of Black Americans \nholding college degrees, Black Americans have higher median \nincomes - especially in larger metro areas",
        x = "Proportion of Black Americans in Metro with College Degree",
        y = "Median Family Income of Black Americans")

## Colors and Scales in `ggplot`

We could spends a great deal of time reviewing how to make changes to colors and scales in `ggplot`. There is nice built-in functionality, and many packages exist to usefully extend what is already there. Take a look at [this site](https://r-graph-gallery.com/color-palette-finder.html) (you may want to bookmark for future reference). In general, there are a range of functions which adjust scales and colors, like `scale_*_continuous`, `scale_color_*`, `scale_fill_` and the like. In general, arguments with `color` change the color of a point or a line, arguments for `fill` change the color of an area. Most arguments are specifically generated for `continuous` or `discrete` data and will use those terms in the argument. Some arguments will end with `_c` for continuous or `_d` for discrete. 

Ok, let's continue refining! We can make the y axis easier to read by presenting the information with commas as a thousands separator and add a dollar sign. Let's also convert our proportion to a percentage in x-axis. The `scales` package lets us access the `label_currency` and `label_percent` functions below. 

In [ ]:
p2 + 
     scale_y_continuous(labels = label_currency(big.mark = ",")) +  # adds commas and dollar signs (default)
     scale_x_continuous(labels = label_percent(scale = 100)) +   # adds percent symbol and rescales by 100x
     theme_minimal() +
     theme(legend.position = "none", 
            plot.title = element_text(size = 14, face = "bold", hjust = 0.5)) +
        labs(title = "In metro areas with a higher percentage of Black Americans \nholding college degrees, Black Americans have higher median \nincomes - especially in larger metro areas",
        x = "Percentage of Black Americans in Metro with College Degree",
        y = "Median Family Income of Black Americans")

There is a great deal of white space here due to a few small outliers with very high median income and high large percentage of Black Americans with BAs. We can utilize our `scale` arguments to restrict the x and y axes.

In [ ]:
p2 + 
     scale_y_continuous(limits = c(0,225000),  # excludes observations > 225k on y
                        labels = label_currency(big.mark = ",")) + 
     scale_x_continuous(limits = c(0,.71),     # excludes observations > .71 on x
                        labels = label_percent(scale = 100)) +
     theme_minimal() +
     theme(legend.position = "none", 
            plot.title = element_text(size = 14, face = "bold", hjust = 0.5)) +
        labs(title = "In metro areas with a higher percentage of Black Americans \nholding college degrees, Black Americans have higher median \nincomes - especially in larger metro areas",
        x = "Percentage of Black Americans in Metro with College Degree",
        y = "Median Family Income of Black Americans")

Finally, let's present one additional variable in this graph: the percentage of the metro area that is Black. To do this, we can use the `color` argument in our original `aes()` call. So we need utilize the full code. Let's store as `p3`.

In [ ]:
p3 <- ggplot(df, aes(   x = ba.black, 
                        y = medfaminc.black, 
                        size = total.pop ,
                        text = NAME,           # we're adding this for plotly later too
                        color = black.pct)) +  # tells R that the color will be based on continuous variable black.pct
        geom_point(alpha = .4) + 
        scale_size_area(max_size = 20) +
        scale_y_continuous(limits = c(0,225000),  
                        labels = label_currency(big.mark = ",")) + 
        scale_x_continuous(limits = c(0,.71),     
                        labels = label_percent(scale = 100)) +
        scale_color_gradient(low = "#6592D6FF", high = "#1E2F46FF", # colors will range from the low to high hex colors
                                limits = c(5, max(df$black.pct))) +     # anything below 5 will be coded gray like NA
        theme_minimal() +
        theme(legend.position = "none", 
            plot.title = element_text(size = 14, face = "bold", hjust = 0.5)) +
        labs(title = "In metro areas with a higher percentage of Black Americans \nholding college  degrees, Black Americans have higher median \nincomes - especially in larger metro areas",
            x = "Percentage of Black Americans in Metro with College Degree",
            y = "Median Family Income of Black Americans")
p3

## Interactive displays using `plotly`

`plotly` is one of several ways to make a data display interactive. I think it is the easiest, although additional work would be need to add an interactive plot, say, to a website. But if you a plot created by `ggplot`, you can convert it to plotly plot for most standard graph types. Simply store your plot as an object and then convert using `ggplotly`.

In [ ]:
# this works because specified the "text" parameter in aes() in the previous plot. 
ggplotly(p3, tooltip = "text") # tooltip tells plotly to present what we placed in the text parameter

Pretty cool, huh? Note that the interactivity won't work inside a pdf document.

## Advanced distributional plots using `ggdist` and `ggdensity`

Often, finding creating ways of describing variation in your data is central to your data visualization efforts. Your audience does not know your data like you do; distributional plots help show the audience your data. Let's examine some more advance plot types using `ggdist` and `ggdensity`.

Before we get started, these plots will be easier if we transform the data one more time.

In [ ]:
medinc <- df |> select(GEOID, NAME, black.count, medfaminc.black, medfaminc.white, pop3) |> # get necessary data
                pivot_longer(cols = starts_with("medfaminc."),  # move data back into rows from coloums
                             names_to = "race",                 # takes racial group names and stores in `race`
                             names_prefix = "medfaminc.",       # takes names from variables with medfaminc. prefix
                             values_to = "medfaminc")           # stores income data in new `medfaminc` variable 

head(medinc)

Notice that we now have two observations per metro, one for "black" and one for "white". `black.count` and `pop3` are coded for both observations inside a metro, but `medfaminc` varies by racial group. And we have a new `race` variable to record which group is being referenced by the median income data. 

`ggdist` has some fantastic options for displaying distributions. All have their place, but I enjoy the `halfeye` plots, which present a distribution and then some measure of central tendency and distribution. Below, we use the `median_qi` which presents the median along with the quartile interval (25% to 75% range).

In [ ]:
ggplot(medinc, aes(x = medfaminc, fill = race)) +
  stat_halfeye(point_interval = "median_qi")      # stat_ instead of geom_ will calculate and graph. 

This is ok, but we can't see the difference between the two distributions very well. We can use the `dodge` position attribute to move the plots vertically. 

In [ ]:
ggplot(medinc, aes(x = medfaminc, fill = race)) +
  stat_halfeye(position = "dodge", point_interval = "median_qi")

Still strange; but we can make this plot better by adding a grouping variable on the y-axis. Let's plot by metro size and add some transparency and labels.

In [ ]:
ggplot(medinc, aes(x = medfaminc, y = pop3, fill = race, labels = pop3)) +
  stat_halfeye(position = "dodge", point_interval = "median_qi") +
  theme_minimal()

We could rearrange these categories. If we group by metro size and make our y-axis set to race, notice how it changes the point of the graph. Below I also call up a different color scheme using the `scale_fill_brewer` function. I also set transparency to differentiate between metro sizes. 

In [ ]:
ggplot(medinc, aes(x = medfaminc, y = race, group = pop3, fill = race, alpha = pop3, labels = pop3)) +
    stat_halfeye(position = "dodge", point_interval = "median_qi") +
    scale_fill_brewer(palette = "Dark2") +
    scale_alpha_manual(values = c(.2, .5, .7)) +
    theme_minimal()

Analyze the previous two graphs in a Markdown chunk. Which do you prefer? How does the organization of the distributions change the point the graph is making?

We could, of course, keep editing scales, colors and titles to improve this graph. Check out the other plot types in `ggdist`. There are many good ones like `dotsinterval` and `swarm`. 

`ggdensity` provides functionality to identify a high-density range (hdr) on a plot, which identifies the area on a two-dimensional plot which contains various proportions of the data.

In [ ]:
ggplot(df |> filter(black.count>1000),   # filter out metros with very few black americans
                aes(x = medfaminc.black, y = medfaminc.white)) + 
  geom_hdr(method = "kde",            # kernal densities for identifying region
            probs = c(.9, .7, .5)) +  # regions for 90%, 70%, and 50% of data
  geom_point(alpha = .1) +            # add scatterplot
  theme_minimal()

We can make some improvements. Let's add a comparison line which will show a 1:1 relationship between median income for white Americans and median income for Black Americans in the metro. We should also make sure the scales are the same on the x and y axis: note that the default graph y-axis range does not include 0! We can also choose a nice color. 

In [ ]:
ggplot(df |> filter(black.count>1000), aes(x = medfaminc.black, y = medfaminc.white)) + 
  geom_hdr(method = "kde", probs = c(.9, .7, .5), fill = "#004949FF") + 
  geom_point(alpha = .1) +
  geom_abline(intercept = 0, slope = 1, color ="#004949FF") + # adds reference line
  expand_limits(x = 0, y = 0) +     # expands scale to include 0 on both axes
  coord_fixed(ratio = 1) +          # set aspect ratio to 1:1
  theme_minimal() 

Much better! And it shows the point that median income in nearly all metros is higher for white Americans than for Black Americans, often substantially so.

## Interactive displays using Datawrapper

There are some nice, free, online programs for creating interactive data displays. Check out [Datawrapper](https://www.datawrapper.de/). I think the easiest way to see what is available and possible is to just start trying to make a plot! I would like you to do the following: First, review what is possible [here](https://www.datawrapper.de/charts/scatter-plot). Then, using your dataframe that includes one observation per metro, calculate the black-white racial income gap (black median income minus white median income). This will be your outcome variable. Identify a potential predictor variable to from your dataframe. Save your data, including your racial income gap variable, the name of the metro area, and your predictor, to a csv file for upload to Datawrapper. You can use `write.csv()`. If you need to, do your own research to figure out how to use this function. Now, make a professional scatterplot in Datawrapper. I want you to show off all that you have learned about making well-designed graphics! Export the data visualization as a png file. Save this png file in the same location in which your .qmd file is saved. Include the png file at the end of your quarto pdf document. Check out how to include a png file in your quarto document [here](https://quarto.org/docs/authoring/figures.html) if needed. Have fun!